In [4]:
from models.resnet.resnet_se_34v2 import ResNetSE34V2
from torchaudio.transforms import MelSpectrogram
from torchaudio.functional import amplitude_to_DB
import torch
import torchaudio
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import yaml
import random

In [2]:
print(os.getcwd())

/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark


In [5]:
DATASET_PATH = "IEMOCAP_full_release"
PATH = os.path.join(os.getcwd(), DATASET_PATH)

In [6]:
data = []

for i, subdir in enumerate(os.listdir(PATH)):
    #if subdir in ["Benjamin_Netanyau", "Julia_Gillard", "Jens_Stoltenberg", "Margaret_Tarcher", "Nelson_Mandela"]:

    if (subdir[0] != '.'):
        #for j in random.sample(range(0, 20), 2):
        #for i, file in enumerate(os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')):
        print(subdir)
        #Choose 2 random file in a directory 
        for j in random.sample(range(0, 20), 2):
            file = os.listdir(os.path.join(PATH, subdir)+'/sentences/wav')[j]
            print(file)
            for k in random.sample(range(0, 20), 3):
                file2 = os.listdir(PATH+'/'+subdir+'/sentences/wav/'+file)[k]
                

                if file2[-8] == 'M' :
                    data.append([PATH+'/'+subdir+'/sentences/wav/'+file+"/"+file2, "Spk_"+str(2*int(subdir[-1])-1)])
                else :
                    data.append([PATH+'/'+subdir+'/sentences/wav/'+file+"/"+file2, "Spk_"+str(2*int(subdir[-1]))])


Session4
Ses04M_script03_1
Ses04M_script01_3
Session3
Ses03F_script02_2
Ses03M_impro07
Session2
Ses02F_script01_1
Ses02M_script03_2
Session5
Ses05F_impro08
Ses05M_script01_1b
Session1
Ses01F_impro06
Ses01M_script01_3


In [7]:
def load_audio(file):
    EPS = 1e-8
    s, _ = librosa.load(file, sr=16000)
    amax = np.max(np.abs(s))
    factor = 1.0 / (amax + EPS)
    s = s * factor
    return s

In [7]:
print(data)

[['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M013.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_F027.wav', 'Spk_8'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script01_3/Ses04F_script01_3_M006.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_1/Ses04F_script02_1_M039.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentences/wav/Ses04F_script02_1/Ses04F_script02_1_M012.wav', 'Spk_7'], ['/Users/hugo/Desktop/Projects/audio_analysis/SpeakerVerification_Benchmark/IEMOCAP_full_release/Session4/sentence

In [8]:
with open('./models/resnet/config.yaml') as f:
    config = yaml.safe_load(f)

sd = torch.load('./models/weigths/resnetse34_epoch92_eer0.00931.pth')
model = ResNetSE34V2(nOut=256, n_mels= 80)
model.load_state_dict(sd)
model.eval()
torch.set_grad_enabled(False)

transform = MelSpectrogram(
    sample_rate= 16000,
    n_fft= 512,
    win_length= 400,
    hop_length= 160,
    window_fn=torch.hamming_window,
    n_mels= 80,
    f_min=20,
    f_max=7600,
    norm='slaney')


def embed_inference(audio_path, transform= transform, model = model):
    s = load_audio(audio_path)
    #print(type(s))
    x = torch.from_numpy(s[None, :])
    x = transform(x)
    x = amplitude_to_DB(
        x, multiplier=10, amin= 1e-5, db_multiplier=0, top_db=75)

    feature = model(x[:, None, :, :])
    feature = torch.nn.functional.normalize(feature)
    return(feature,s)

Embedding size is 256, encoder SAP.


In [9]:
loss = torch.nn.CosineSimilarity(dim=1, eps=1e-8)

# PROCESS THE TEST

In [10]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

In [12]:
from modelscope.pipelines import pipeline

sv_pipeline = pipeline(
    task='speaker-verification',
    model='damo/speech_campplus_sv_en_voxceleb_16k'
)

2023-10-10 11:40:04,455 - modelscope - INFO - PyTorch version 2.1.0 Found.
2023-10-10 11:40:04,457 - modelscope - INFO - Loading ast index from /Users/hugo/.cache/modelscope/ast_indexer
2023-10-10 11:40:04,499 - modelscope - INFO - No valid ast index found from /Users/hugo/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2023-10-10 11:40:04,551 - modelscope - INFO - Loading done! Current index file version is 1.9.2, with md5 95894f5bac47d08b4fba17db5dc4d12a and a total number of 941 components indexed
2023-10-10 11:40:18,133 - modelscope - INFO - Model revision not specified, use revision: v1.0.2
2023-10-10 11:40:19,113 - modelscope - INFO - initiate model from /Users/hugo/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k
2023-10-10 11:40:19,116 - modelscope - INFO - initiate model from location /Users/hugo/.cache/modelscope/hub/damo/speech_campplus_sv_en_voxceleb_16k.
2023-10-10 11:40:19,120 - modelscope - INFO - initialize model from /Users/hugo/.cache/m

In [13]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")


In [14]:

compar_tab = pd.DataFrame(columns = ['spk_1', 'spk_2', 'duration_1', 'duration_2', 'score_classic', 'score_CAM++', 'TDN_score','path_1', 'path_2'])
with tqdm(total=len(data)) as pbar:
    for sample_1 in data :
        spk_1 = sample_1[1]
        ref_embed, audio_1 = embed_inference(sample_1[0])
        for sample_2 in data :
            spk_2 = sample_2[1]

            
            embed, audio_2 = embed_inference(sample_2[0])
            loss_val = loss(ref_embed, embed)



            cam_score = sv_pipeline([audio_1, audio_2])

            TDN_score = verification.verify_files(sample_1[0], sample_2[0]) # Different Speakers
            compar_tab.loc[len(compar_tab)] = [spk_1, spk_2, len(audio_1)/16000, len(audio_2)/16000, loss_val.item(), cam_score, TDN_score, sample_1[0], sample_2[0]]
            #print(label, val)
        pbar.update(1)
       

           
compar_tab.to_csv('compar_tab.csv', index=False)

100%|██████████| 30/30 [23:02<00:00, 46.08s/it]


In [12]:
ref_embed, audio = embed_inference(data[0][0])

In [16]:
print(len(audio)/16000)

9.4199375


In [42]:
df = pd.read_csv('compar_tab.csv')

false_positive_classic = 0
false_positive_CAM = 0
false_negative_classic = 0
false_negative_CAM = 0
false_positive_TDN = 0
false_negative_TDN = 0
positive = 0
negative = 0
treshold = 0.3

for treshold in [0.2,0.3,0.4,0.5,0.6,0.7] :
    print('********************** CHOSEN TRESHOLD : ', treshold, ' **********************')
    for row, line in df.iterrows():
        score_classic = line['score_classic']
    
        CAM = str_to_dict_cam(line['score_CAM++'])
        score_CAM = float(CAM['score'])
        
        TDN = str_to_dict_TDN(line['TDN_score'])
        score_TDN = float(TDN[0])

        if line['spk_1'] == line['spk_2'] :
            positive += 1
            if score_classic < treshold :
                false_positive_classic += 1
            if score_CAM < treshold :
                false_positive_CAM += 1
            if score_TDN < treshold :
                false_positive_TDN += 1
        else :
            negative += 1
            if score_classic > treshold :
                false_negative_classic += 1
            if score_CAM > treshold :
                false_negative_CAM += 1
            if score_TDN > treshold :
                false_negative_TDN += 1

        
    print(f'For RESNET : False positive : {false_positive_classic} False negative : {false_negative_classic}')
    print(f'For CAM++ : False positive : {false_positive_CAM} False negative : {false_negative_CAM}')
    print(f'For TDN : False positive : {false_positive_TDN} False negative : {false_negative_TDN}')
    print(f'For RESNET : Accuracy : {(positive-false_positive_classic)/positive}')
    print(f'For CAM++ : Accuracy : {(positive-false_positive_CAM)/positive}')   
    print(f'For TDN : Accuracy : {(positive-false_positive_TDN)/positive}')

    false_positive_classic = 0
    false_positive_CAM = 0
    false_negative_classic = 0
    false_negative_CAM = 0
    false_positive_TDN = 0
    false_negative_TDN = 0
    positive = 0
    negative = 0


********************** CHOSEN TRESHOLD :  0.2  **********************
For RESNET : False positive : 14 False negative : 144
For CAM++ : False positive : 8 False negative : 80
For TDN : False positive : 16 False negative : 40
For RESNET : Accuracy : 0.8833333333333333
For CAM++ : Accuracy : 0.9333333333333333
For TDN : Accuracy : 0.8666666666666667
********************** CHOSEN TRESHOLD :  0.3  **********************
For RESNET : False positive : 40 False negative : 46
For CAM++ : False positive : 24 False negative : 12
For TDN : False positive : 34 False negative : 4
For RESNET : Accuracy : 0.6666666666666666
For CAM++ : Accuracy : 0.8
For TDN : Accuracy : 0.7166666666666667
********************** CHOSEN TRESHOLD :  0.4  **********************
For RESNET : False positive : 60 False negative : 6
For CAM++ : False positive : 52 False negative : 0
For TDN : False positive : 66 False negative : 0
For RESNET : Accuracy : 0.5
For CAM++ : Accuracy : 0.5666666666666667
For TDN : Accuracy : 0.4

In [15]:
text = '(tensor([1.0000]), tensor([True]))'

In [45]:
test = "{'score':1.0 , 'text':'hello'}"

In [32]:
# How to change a str written like {'score':1.0 , 'text':'hello'} to a dict
def str_to_dict_cam(input : str):
    input = input.replace(' ', '')
    input = input.replace('{', '')
    input = input.replace('}', '')
    input = input.replace("'", '')
    input = input.split(',')
    output = {}
    for i in input :
        i = i.split(':')
        output[i[0]] = i[1]
    return output

In [36]:
def str_to_dict_TDN(input : str):
    input = input.replace('tensor', '')
    input = input.replace(' ', '')
    input = input.replace('{', '')
    input = input.replace('}', '')
    input = input.replace("'", '')
    input = input.replace("(", '')
    input = input.replace(")", '')
    input = input.replace('[', '')
    input = input.replace(']', '')

    output = input.split(',')

    return output

In [30]:
print(str_to_dict_TDN(text))

1.0000,True
['1.0000', 'True']
